In [42]:
%pip install sqlalchemy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: c:\Users\jason\.pyenv\pyenv-win\versions\3.13.0\python.exe -m pip install --upgrade pip


In [ ]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd

# Database connection info (update with your info)
username = ""
password = ""
server = ""
database = ""

CONNECTION_STRING = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

# Create a SQLAlchemy engine
engine = create_engine(CONNECTION_STRING)
metadata = MetaData()

# Reflect tables
headstart_table = Table("headstart_wa", metadata, autoload_with=engine)
ncesdata_table = Table("ncesdata_2D86566_wa", metadata, autoload_with=engine)

# Fetch records using SQLAlchemy table reflection
with engine.connect() as conn:
    headstart_df = pd.DataFrame(conn.execute(headstart_table.select()).fetchall(), columns=headstart_table.columns.keys())
    ncesdata_df = pd.DataFrame(conn.execute(ncesdata_table.select()).fetchall(), columns=ncesdata_table.columns.keys())

# Column cleanup
for column in ncesdata_df.columns:
    ncesdata_df[column] = ncesdata_df[column].astype(str).fillna('')
#ncesdata_unique = ncesdata_df.drop_duplicates(subset=["Street_Address", "City", "State", "ZIP", "ZIP_4_digit"])
ncesdata_unique = ncesdata_df.drop_duplicates(subset=["Street_Address", "City", "State", "ZIP"]) # This does not quite line up with SQL, off by 2

headstart_df["zipFour"] = headstart_df["zipFour"].apply(lambda x: None if x < 1 else x)
for column in headstart_df.columns:
    headstart_df[column] = headstart_df[column].astype(str).fillna('')
headstart_unique = headstart_df.drop_duplicates(subset=["addressLineOne", "addressLineTwo", "city", "state", "zipFive", "zipFour", "county"])

# Count unique addresses
result = pd.DataFrame({
    "dataset_name": ["headstart", "ncesdata"],
    "unique_addresses": [len(headstart_unique), len(ncesdata_unique)]
})

print(result)

  dataset_name  unique_addresses
0    headstart               320
1     ncesdata              2233


In [ ]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd

# Database connection info (update with your info)
username = ""
password = ""
server = ""
database = ""

CONNECTION_STRING = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

# Create a SQLAlchemy engine
engine = create_engine(CONNECTION_STRING)
metadata = MetaData()

# Reflect tables
headstart_table = Table("headstart_wa", metadata, autoload_with=engine)
ncesdata_table = Table("ncesdata_2D86566_wa", metadata, autoload_with=engine)

# Fetch records using SQLAlchemy table reflection
with engine.connect() as conn:
    headstart_df = pd.DataFrame(conn.execute(headstart_table.select()).fetchall(), columns=headstart_table.columns.keys())
    ncesdata_df = pd.DataFrame(conn.execute(ncesdata_table.select()).fetchall(), columns=ncesdata_table.columns.keys())


headstart_df = headstart_df.rename(columns={
    "addressLineOne": "street_adr",
    "city": "city_adr",
    "state": "state_adr",
    "zipFive": "zip_adr",
    "zipFour": "zip_4_digit_adr"
})

ncesdata_df = ncesdata_df.rename(columns={
    "Street_Address": "street_adr",
    "City": "city_adr",
    "State": "state_adr",
    "ZIP": "zip_adr",
    "ZIP_4_digit": "zip_4_digit_adr"
})

# Column cleanup
headstart_df["zip_4_digit_adr"] = headstart_df["zip_4_digit_adr"].apply(lambda x: None if x < 1 else x)

for column in headstart_df.columns:
    headstart_df[column] = headstart_df[column].astype(str).fillna('')

for column in ncesdata_df.columns:
    ncesdata_df[column] = ncesdata_df[column].astype(str).fillna('')

# Remove duplicates
headstart_unique = headstart_df.drop_duplicates(subset=["street_adr", "city_adr", "state_adr", "zip_adr", "zip_4_digit_adr"])
#ncesdata_unique = ncesdata_df.drop_duplicates(subset=["street_adr", "city_adr", "state_adr", "zip_adr", "zip_4_digit_adr"])
ncesdata_unique = ncesdata_df.drop_duplicates(subset=["street_adr", "city_adr", "state_adr", "zip_adr"]) # This does not quite line up with SQL, off by 2

# Join and count overlapping records
overlap_count = headstart_unique.merge(
    ncesdata_unique, 
    #on=["street_adr", "city_adr", "state_adr", "zip_adr", "zip_4_digit_adr"],
    on=["street_adr", "city_adr", "state_adr", "zip_adr"],
    #on=["street_adr", "city_adr", "state_adr"],
    how="inner"
).shape[0]

print(f"Number of overlapping addresses: {overlap_count}")


Number of overlapping addresses: 37


In [ ]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd

# Database connection info (update with your info)
username = ""
password = ""
server = ""
database = ""

CONNECTION_STRING = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

# Create a SQLAlchemy engine
engine = create_engine(CONNECTION_STRING)
metadata = MetaData()

# Reflect tables
headstart_table = Table("headstart_wa", metadata, autoload_with=engine)
ncesdata_table = Table("ncesdata_2D86566_wa", metadata, autoload_with=engine)

# Fetch records using SQLAlchemy table reflection
with engine.connect() as conn:
    headstart_df = pd.DataFrame(conn.execute(headstart_table.select()).fetchall(), columns=headstart_table.columns.keys())
    ncesdata_df = pd.DataFrame(conn.execute(ncesdata_table.select()).fetchall(), columns=ncesdata_table.columns.keys())

# Apply transformations for headstart_wa
headstart_df = headstart_df.rename(columns={
    "addressLineOne": "street_adr",
    "city": "city_adr",
    "state": "state_adr",
    "zipFive": "zip_adr",
    "zipFour": "zip_4_digit_adr"
})
headstart_df["zip_4_digit_adr"] = headstart_df["zip_4_digit_adr"].apply(lambda x: None if x < 1 else x)

# Apply transformations for ncesdata_2D86566_wa
ncesdata_df = ncesdata_df.rename(columns={
    "Street_Address": "street_adr",
    "City": "city_adr",
    "State": "state_adr",
    "ZIP": "zip_adr",
    "ZIP_4_digit": "zip_4_digit_adr"
})

for column in headstart_df.columns:
    headstart_df[column] = headstart_df[column].astype(str).fillna('')

for column in ncesdata_df.columns:
    ncesdata_df[column] = ncesdata_df[column].astype(str).fillna('')

# Remove duplicates
headstart_unique = headstart_df.drop_duplicates(subset=["street_adr", "city_adr", "state_adr", "zip_adr", "zip_4_digit_adr"])
#ncesdata_unique = ncesdata_df.drop_duplicates(subset=["street_adr", "city_adr", "state_adr", "zip_adr", "zip_4_digit_adr"])
ncesdata_unique = ncesdata_df.drop_duplicates(subset=["street_adr", "city_adr", "state_adr", "zip_adr"]) # This does not quite line up with SQL, off by 2

# Perform left join and create match indicator
merged_df = headstart_df.merge(
    ncesdata_df,
    #on=["street_adr", "city_adr", "state_adr", "zip_adr", "zip_4_digit_adr"],
    #on=["street_adr", "city_adr", "state_adr", "zip_adr"],
    on=["street_adr", "city_adr", "state_adr"],
    how="left",
    indicator=True
)

# Add match indicator column based on _merge values
merged_df["is_match_indicator"] = merged_df["_merge"].apply(lambda x: "true" if x == "both" else "false")

# Drop the _merge column and sort results
merged_df = merged_df.drop(columns=["_merge"]).sort_values(by=["is_match_indicator", "street_adr"], ascending=[False, True])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(merged_df)
#merged_df.to_csv("merged.csv", index=False)

                                                  name  \
137                                               Mann   
126                                         Washington   
177                                    CLOVER PARK ELP   
226                                        NVCCF-Sumas   
355                                         Parkway HS   
131                                            Whitman   
311       Walla Walla Center for Children and Families   
312       Walla Walla Center for Children and Families   
148                                             Birney   
167                               BETHEL EVERGREEN EHS   
139                                           Franklin   
198          Port Townsend Head Start/Early Head Start   
25                             RENTON MEADOW CREST ELC   
113                                   Jefferson Center   
306                                                SCC   
307                                        SCC Bigfoot   
127           